<a href="https://colab.research.google.com/github/daferocu/Topicos-Avanzados/blob/main/datalab/Talleres%202/E9%20-%20Question%20Answering/E9_QuestionAnswer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question & Answer

Creating a Question-Answer Transformer model or QA Transformer can be beneficial for several reasons, particularly in the field of Natural Language Processing (NLP). Here are some compelling reasons why you might want to develop a QA Transformer:

1. **Question-Answering Systems:** QA Transformers are designed to provide accurate and contextually relevant answers to questions posed in natural language. These systems have a wide range of practical applications, including chatbots, virtual assistants, customer support, and information retrieval.

2. **Information Retrieval:** QA Transformers can be used to search through large corpora of text and extract precise answers to user queries. This can improve the efficiency and effectiveness of information retrieval systems.

3. **Document Summarization:** QA Transformers can be used to summarize long documents by answering questions about the document's content. This makes it easier for users to quickly understand the key points and relevant information in a text.

4. **Education and E-Learning:** QA Transformers can be integrated into educational platforms to provide instant answers and explanations to students' questions. They can also help with the automatic generation of quiz questions and answers.

5. **Content Generation:** QA Transformers can assist in content generation by automatically answering questions based on available knowledge. This can be useful for generating FAQs, product descriptions, and informative articles.

6. **Customer Support:** Many companies use QA systems to automate responses to frequently asked questions, freeing up human agents to handle more complex queries and providing customers with quick solutions.

7. **Medical Diagnosis:** QA Transformers can assist medical professionals by answering questions related to patient records, medical literature, and diagnostic information, potentially leading to faster and more accurate diagnoses.

8. **Legal and Compliance:** In the legal field, QA Transformers can be used to search and extract information from legal documents, assisting lawyers in their research and case preparation.

9. **Language Translation:** QA Transformers can be used to answer questions about language translation, helping users understand the meaning of words, phrases, or sentences in different languages.

10. **Scientific Research:** QA Transformers can support researchers by answering questions related to scientific literature, allowing them to quickly access relevant information for their studies.

11. **Decision Support:** QA Transformers can aid in decision-making processes by providing answers to questions related to data analysis, market research, and business intelligence.

12. **Accessibility:** QA Transformers can improve accessibility for individuals with disabilities by providing spoken or written answers to their questions, helping them access information more easily.

Overall, QA Transformers have the potential to enhance information retrieval, automation, and user interaction in various domains, making them a valuable tool in the development of intelligent systems and applications. The ability to provide accurate and context-aware answers to questions in natural language is a key advantage of these models.

---
Exercise:

Now, as a data scientist expert in NLP, you are asked to create a model to be able to answer question in Spanish. Your stakeholders will pass you an article and one question and your model should answer it.

In [17]:
!pip install requests beautifulsoup4
!pip install transformers
!pip install sentencepiece

In [18]:
# Importamos funciones de NLTK para tokenizar el texto en oraciones y palabras
from nltk.tokenize import sent_tokenize, word_tokenize

# Importamos la clase PorterStemmer de NLTK para hacer stemming (reducir palabras a su raíz)
from nltk.stem import PorterStemmer

# Importamos TfidfVectorizer de scikit-learn para convertir el texto en vectores basados en TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Importamos la función cosine_similarity de scikit-learn para medir la similitud entre vectores
from sklearn.metrics.pairwise import cosine_similarity

# Importamos las stopwords de NLTK, que son palabras comunes que pueden ser ignoradas en análisis de texto
from nltk.corpus import stopwords

# Importamos la librería string, que contiene funciones y constantes para manipular texto
import string

# Importamos requests para hacer peticiones HTTP y descargar contenido de sitios web
import requests

# Importamos BeautifulSoup de bs4 para analizar (parsear) el HTML y extraer información de páginas web
from bs4 import BeautifulSoup

# Importamos el tokenizador y el modelo de BERT para tareas de pregunta-respuesta
from transformers import BertTokenizer, BertForQuestionAnswering

# Importamos torch para trabajar con tensores y manejar modelos de deep learning en PyTorch
import torch

# Importamos pipeline de transformers, que nos permite ejecutar tareas de NLP de manera sencilla
from transformers import pipeline

# Importamos pandas, una librería para manipular y analizar datos tabulares en estructuras como DataFrames
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Extraemos el contenido de texto que tiene la URL, usamos la solicitud HTTP get de la biblioteca requests para obtener el código de la pagina. Si devuelve 200 quiere decir que la solicitud fue exitosa. Ahora con la biblioteca BeautifulSoup, que facilita la manipulacion dentro de los documentos HTML. El codigo busca una división con la clase "article-content", donde generalmente se encuentra el cuerpo del artículo. A continuación, extrae todo el texto dentro de las etiquetas "p" (que suelen representar párrafos) y lo agrega a una cadena article_text.

In [19]:
# URL del artículo
url = "https://time.com/collection/time100-ai/6309026/geoffrey-hinton/"

# Realizar una solicitud HTTP para obtener el contenido de la página
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    # Analizar el contenido HTML de la página con BeautifulSoup
    soup = BeautifulSoup(response.text, "html.parser")

    # Encontrar el contenido del artículo
    article_content = soup.find("div", {"class": "article-content"})

    # Extraer el texto del artículo
    article_text = ""
    for paragraph in article_content.find_all("p"):
        article_text += paragraph.get_text() + "\n"

    # Imprimir el texto del artículo
    print(article_text)
else:
    print("Error al obtener la página:", response.status_code)

Over the course of February, Geoffrey Hinton, one of the most influential AI researchers of the past 50 years, had a “slow eureka moment.”
Hinton, 76, has spent his career trying to build AI systems that model the human brain, mostly in academia before joining Google in 2013. He had always believed that the brain was better than the machines that he and others were building, and that by making them more like the brain, they would improve. But in February, he realized “the digital intelligence we’ve got now may be better than the brain already. It’s just not scaled up quite as big.” 
Developers around the world are currently racing to build the biggest AI systems that they can. Given the current rate at which AI companies are increasing the size of models, it could be less than five years until AI systems have 100 trillion connections—roughly as many as there are between neurons in the human brain.
Alarmed, Hinton left his post as VP and engineering fellow in May and gave a flurry of in

In [20]:
question = "How is Geoffrey Hinton?"

Realizamos la implementación un sistema básico de TF-IDF (Term Frequency-Inverse Document Frequency) combinado con similitud de coseno para encontrar la oración más relevante de un texto en relación a una pregunta dada. Primero, el texto (extraído previamente) y la pregunta se tokenizan y preprocesan eliminando puntuaciones, palabras vacías (stopwords) y aplicando stemming (reducción de palabras a su raíz). Luego, se utiliza el vectorizador TF-IDF para crear una matriz que representa las oraciones del texto y la pregunta. Posteriormente, se calcula la similitud de coseno entre la pregunta y cada oración del texto. La oración con la mayor similitud es seleccionada como la respuesta a la pregunta.

In [21]:
# Define el texto y la pregunta
texto = article_text
question = "How is Geoffrey Hinton?"

In [22]:
# Tokeniza el texto en oraciones
oraciones = sent_tokenize(texto)

In [23]:
# Función de preprocesamiento
def preprocesar(text):
    # Tokeniza el texto en palabras
    palabras = word_tokenize(text)

    # Convierte todas las palabras a minúsculas y elimina los signos de puntuación
    palabras = [word.lower() for word in palabras if word not in string.punctuation]

    # Elimina las palabras vacías (stopwords) del texto
    palabras = [word for word in palabras if word not in stopwords.words('english')]

    # Inicializa el algoritmo de stemming de Porter, ayuda a normalizar palabras reduciéndolas a su forma base, como por ejemplo convertir “running” y “runs” en “run”.
    stemmer = PorterStemmer()

    # Aplica el stemming a cada palabra (reduce las palabras a su raíz)
    palabras = [stemmer.stem(word) for word in palabras]

    # Une las palabras procesadas en una cadena de texto
    return " ".join(palabras)

In [24]:
# Preprocesa todas las oraciones
oraciones_preprocesadas = [preprocesar(oracion) for oracion in oraciones]
# Preprocesa la pregunta
pregunta_preprocesada = preprocesar(question)

In [25]:
# Crea la matriz TF-IDF para las oraciones y la pregunta
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(oraciones_preprocesadas + [pregunta_preprocesada])

In [26]:
# Calcula la similitud de coseno entre la pregunta y todas las oraciones
similarity_scores = cosine_similarity(tfidf_matrix[-1:], tfidf_matrix[:-1]).flatten()

In [27]:
print(similarity_scores)

[0.25227034 0.         0.         0.         0.         0.04120537
 0.         0.08619847 0.06203478 0.11479562 0.         0.
 0.03991913 0.         0.05394728 0.         0.06160146 0.
 0.04403329 0.         0.13223232 0.         0.         0.08328896
 0.         0.04042246 0.         0.09689021 0.         0.
 0.         0.04407544 0.         0.         0.06863848 0.04646759
 0.         0.         0.05832075 0.        ]


In [28]:
# Encuentra el índice de la oración más similar
most_similar_sentence_index = similarity_scores.argmax()

# Obtiene la oración correspondiente como respuesta
respuesta = oraciones[most_similar_sentence_index]
print("Pregunta:", question)
print("Respuesta:", respuesta)

Pregunta: How is Geoffrey Hinton?
Respuesta: Over the course of February, Geoffrey Hinton, one of the most influential AI researchers of the past 50 years, had a “slow eureka moment.”
Hinton, 76, has spent his career trying to build AI systems that model the human brain, mostly in academia before joining Google in 2013.


Utilizamos el modelo BERT preentrenado para la tarea de respuesta a preguntas. Incluimos el token de Hugging Face para acceder al modelo. Luego, carga el modelo BERT (bert-large-uncased-whole-word-masking-finetuned-squad) y su correspondiente tokenizador. A continuación, se prepara la entrada para el modelo, combinando la pregunta y el texto en un formato adecuado y aplicando truncamiento si el texto excede los 512 tokens. Durante la inferencia, el modelo predice los logits de inicio y fin de la respuesta en el texto. Finalmente, se extraen los tokens correspondientes a la respuesta usando los índices predichos, y se decodifican para obtener la respuesta en formato legible.

In [ ]:
from google.colab import output
import getpass

# Pedir el token al usuario
HF_TOKEN = getpass.getpass('Introduce tu Hugging Face token:')

Introduce tu Hugging Face token:··········


In [29]:
# Cargar el modelo BERT y el tokenizador
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForQuestionAnswering.from_pretrained(model_name)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
# Preparar la entrada para el modelo
inputs = tokenizer.encode_plus(question, texto, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512)


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [31]:
# Realizar la inferencia
with torch.no_grad():
    outputs = model(**inputs)

# Extraer los índices de inicio y fin de la respuesta
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1

In [32]:
answer_tokens = inputs["input_ids"][0][answer_start:answer_end]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)
print("Respuesta:", answer)

Respuesta: one of the most influential ai researchers of the past 50 years


# Transformers Question Answer

En esta sección realizamos tareas de procesamiento utilizando pipelines de la biblioteca Transformers:



1.   **Clasificación de texto:** Se usa un pipeline de clasificación de
texto con el modelo distilbert-base-uncased-finetuned-sst-2-english. Se clasifica el artículo (article_text) que ha sido truncado a 512 caracteres. El resultado de la clasificación se muestra en un DataFrame para facilitar su análisis.
2.   **Respuesta a preguntas:** Se utiliza un pipeline de pregunta-respuesta (question-answering) con el modelo deepset/roberta-base-squad2, especificando el parámetro clean_up_tokenization_spaces para evitar futuros problemas de tokenización. Se le hace una pregunta en español sobre Geoffrey Hinton utilizando el contenido de article_text como contexto, y la respuesta se muestra también en un DataFrame.
3.   **Resumen de texto:** Se utiliza un pipeline de resumen con el modelo facebook/bart-large-cnn para generar un resumen del texto, limitando el resultado a 60 tokens. Se asegura de limpiar los espacios de tokenización durante el proceso y finalmente se imprime el resumen generado.


In [62]:
# Importa la función 'pipeline' de la librería 'transformers'
from transformers import pipeline

# Crea un clasificador de texto usando el pipeline de 'text-classification'
classifier = pipeline("text-classification", model="distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [47]:
article_text = article_text[:512]

outputs = classifier(article_text)
pd.DataFrame(outputs)

,label,score
0,POSITIVE,0.787145


In [60]:
# Especificar el modelo para pregunta-respuesta y establecer el parámetro 'clean_up_tokenization_spaces'
reader = pipeline(
    "question-answering",
    model="deepset/roberta-base-squad2",
    clean_up_tokenization_spaces=True
)
#pregunta en español
question = "¿Que ha hecho Geoffrey Hinton?"
outputs = reader(question=question, context=article_text)
pd.DataFrame([outputs])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


,score,start,end,answer
0,0.196659,118,136,slow eureka moment


In [61]:
# Importa el pipeline de resumen y especifica el modelo que deseas usar para evitar usar el modelo por defecto
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", clean_up_tokenization_spaces=True)

# Generar el resumen con un límite de longitud de 60 tokens y asegurarse de que se limpie correctamente la tokenización
outputs = summarizer(article_text, max_length=60, clean_up_tokenization_spaces=True)

# Imprimir el resumen generado
print(outputs[0]['summary_text'])

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Geoffrey Hinton, 76, has spent his career trying to build AI systems that model the human brain. He had always believed that the brain was better than the machines that he and others were building. But in February, he realized “the digital intelligence we’ve


In [53]:
print (article_text)

Over the course of February, Geoffrey Hinton, one of the most influential AI researchers of the past 50 years, had a “slow eureka moment.”
Hinton, 76, has spent his career trying to build AI systems that model the human brain, mostly in academia before joining Google in 2013. He had always believed that the brain was better than the machines that he and others were building, and that by making them more like the brain, they would improve. But in February, he realized “the digital intelligence we’ve got now 


# Traducción texto con HugginFace

Este código carga un modelo preentrenado de MarianMT para traducir texto del inglés al español. Utiliza el tokenizador para preparar el texto en inglés y genera su traducción al español mediante el modelo. Luego, decodifica los tokens traducidos y muestra el artículo traducido en español, para finalmente realizar la pregunta y respuesta en español.

In [54]:
from transformers import MarianMTModel, MarianTokenizer

# Cargar el modelo y el tokenizador para traducción inglés-español
model_name = "Helsinki-NLP/opus-mt-en-es"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Función para traducir texto del inglés al español
def traducir(texto):
    # Tokeniza el texto de entrada
    inputs = tokenizer(texto, return_tensors="pt", truncation=True, padding="longest")
    # Genera la traducción
    translated_tokens = model.generate(**inputs)
    # Decodifica los tokens traducidos
    traduccion = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return traduccion

# Traducir el artículo al español
articulo_traducido = traducir(article_text)

print("Artículo traducido:", articulo_traducido)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Artículo traducido: A lo largo de febrero, Geoffrey Hinton, uno de los investigadores de IA más influyentes de los últimos 50 años, tuvo un “momento eureka lento.” Hinton, de 76 años, ha pasado su carrera tratando de construir sistemas de IA que modelen el cerebro humano, sobre todo en la academia antes de unirse a Google en 2013. Siempre había creído que el cerebro era mejor que las máquinas que él y otros estaban construyendo, y que al hacerlos más como el cerebro, mejorarían. Pero en febrero, se dio cuenta de “la inteligencia digital que tenemos ahora


In [59]:
# Creación de un pipeline de respuesta a preguntas en español
reader = pipeline("question-answering", model="deepset/roberta-base-squad2")

question = "¿Quien es Geoffrey Hinton?"
outputs = reader(question=question, context=articulo_traducido)

# Mostrar la respuesta a la pregunta en español
print("Respuesta:", outputs['answer'])

Respuesta: uno de los investigadores de IA más influyentes
